In [1]:
from main import *

In [2]:

now = datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S")

# add cwd for convenience and to make classes in this file available when
# running as `python main.py`
# (in particular `main.DataModuleFromConfig`)
sys.path.append(os.getcwd())

In [3]:
parser = get_parser()
parser = Trainer.add_argparse_args(parser)

In [4]:
opt, unknown = parser.parse_known_args()

In [5]:
opt

Namespace(accelerator=None, accumulate_grad_batches=None, amp_backend='native', amp_level=None, auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, base=[], benchmark=None, check_val_every_n_epoch=1, checkpoint_callback=None, debug=False, default_root_dir=None, detect_anomaly=False, deterministic=False, devices=None, enable_checkpointing=True, enable_model_summary=True, enable_progress_bar=True, fast_dev_run=False, flush_logs_every_n_steps=None, gpus=None, gradient_clip_algorithm=None, gradient_clip_val=None, ipus=None, limit_predict_batches=None, limit_test_batches=None, limit_train_batches=None, limit_val_batches=None, log_every_n_steps=50, log_gpu_memory=None, logdir='logs', logger=True, max_epochs=None, max_steps=-1, max_time=None, min_epochs=None, min_steps=None, move_metrics_to_cpu=False, multiple_trainloader_mode='max_size_cycle', name='', no_test=False, num_nodes=1, num_processes=None, num_sanity_val_steps=2, overfit_batches=0.0, plugins=None, postfix='C:\\

In [6]:
opt.base = ['configs/latent-diffusion/lsun_churches-ldm-kl-8.yaml']
opt.gpus = 1

In [7]:
opt.postfix = ""

In [8]:
if opt.name:
    name = "_" + opt.name
elif opt.base:
    cfg_fname = os.path.split(opt.base[0])[-1]
    cfg_name = os.path.splitext(cfg_fname)[0]
    name = "_" + cfg_name
else:
    name = ""
nowname = now + name + opt.postfix
logdir = os.path.join(opt.logdir, nowname)

ckptdir = os.path.join(logdir, "checkpoints")
cfgdir = os.path.join(logdir, "configs")
seed_everything(opt.seed)

Global seed set to 23


23

In [9]:
print(nowname)

2022-06-17T14-39-19_lsun_churches-ldm-kl-8


In [10]:
# pause

In [11]:
# init and save configs
configs = [OmegaConf.load(cfg) for cfg in opt.base]

In [12]:
cli = OmegaConf.from_dotlist(unknown)
config = OmegaConf.merge(*configs, cli)
lightning_config = config.pop("lightning", OmegaConf.create())
# merge trainer cli with config
trainer_config = lightning_config.get("trainer", OmegaConf.create())

In [13]:
# default to ddp
trainer_config["accelerator"] = "gpu"
trainer_config["devices"] = 1

for k in nondefault_trainer_args(opt):
    trainer_config[k] = getattr(opt, k)
if not "gpus" in trainer_config:
    del trainer_config["accelerator"]
    cpu = True
else:
    gpuinfo = trainer_config["gpus"]
    print(f"Running on GPUs {gpuinfo}")
    cpu = False
    
trainer_opt = argparse.Namespace(**trainer_config)
lightning_config.trainer = trainer_config

Running on GPUs 1


In [14]:
config.model['params']['timesteps'] = 20

In [15]:
# model
model = instantiate_from_config(config.model)

LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 294.97 M params.
Keeping EMAs of 522.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Restored from models/first_stage_models/kl-f8/model.ckpt
Training LatentDiffusion as an unconditional model.


In [16]:
trainer_config

{'benchmark': True, 'accelerator': 'gpu', 'devices': 1, 'gpus': 1}

In [18]:
#trainer.model

In [19]:
print(model.device)

cpu


In [20]:
pause

NameError: name 'pause' is not defined

In [21]:
model.betas

tensor([0.0015, 0.0019, 0.0023, 0.0027, 0.0032, 0.0038, 0.0043, 0.0049, 0.0056,
        0.0063, 0.0070, 0.0078, 0.0086, 0.0095, 0.0104, 0.0113, 0.0123, 0.0133,
        0.0144, 0.0155])

In [22]:
# Train

In [23]:
# trainer and callbacks
trainer_kwargs = dict()

In [24]:
# default logger configs
default_logger_cfgs = {
    "wandb": {
        "target": "pytorch_lightning.loggers.WandbLogger",
        "params": {
            "name": nowname,
            "save_dir": logdir,
            "offline": opt.debug,
            "id": nowname,
        }
    },
    "testtube": {
        "target": "pytorch_lightning.loggers.TestTubeLogger",
        "params": {
            "name": "testtube",
            "save_dir": logdir,
        }
    },
}
default_logger_cfg = default_logger_cfgs["testtube"]
if "logger" in lightning_config:
    logger_cfg = lightning_config.logger
else:
    logger_cfg = OmegaConf.create()
logger_cfg = OmegaConf.merge(default_logger_cfg, logger_cfg)
trainer_kwargs["logger"] = instantiate_from_config(logger_cfg)

# modelcheckpoint - use TrainResult/EvalResult(checkpoint_on=metric) to
# specify which metric is used to determine best models
default_modelckpt_cfg = {
    "target": "pytorch_lightning.callbacks.ModelCheckpoint",
    "params": {
        "dirpath": ckptdir,
        "filename": "{epoch:06}",
        "verbose": True,
        "save_last": True,
    }
}
if hasattr(model, "monitor"):
    print(f"Monitoring {model.monitor} as checkpoint metric.")
    default_modelckpt_cfg["params"]["monitor"] = model.monitor
    default_modelckpt_cfg["params"]["save_top_k"] = 3

if "modelcheckpoint" in lightning_config:
    modelckpt_cfg = lightning_config.modelcheckpoint
else:
    modelckpt_cfg =  OmegaConf.create()
modelckpt_cfg = OmegaConf.merge(default_modelckpt_cfg, modelckpt_cfg)
print(f"Merged modelckpt-cfg: \n{modelckpt_cfg}")
if version.parse(pl.__version__) < version.parse('1.4.0'):
    trainer_kwargs["checkpoint_callback"] = instantiate_from_config(modelckpt_cfg)

# add callback which sets up log directory
default_callbacks_cfg = {
    "setup_callback": {
        "target": "main.SetupCallback",
        "params": {
            "resume": opt.resume,
            "now": now,
            "logdir": logdir,
            "ckptdir": ckptdir,
            "cfgdir": cfgdir,
            "config": config,
            "lightning_config": lightning_config,
        }
    },
    "image_logger": {
        "target": "main.ImageLogger",
        "params": {
            "batch_frequency": 750,
            "max_images": 4,
            "clamp": True
        }
    },
    "learning_rate_logger": {
        "target": "main.LearningRateMonitor",
        "params": {
            "logging_interval": "step",
            # "log_momentum": True
        }
    },
    "cuda_callback": {
        "target": "main.CUDACallback"
    },
}
if version.parse(pl.__version__) >= version.parse('1.4.0'):
    default_callbacks_cfg.update({'checkpoint_callback': modelckpt_cfg})

if "callbacks" in lightning_config:
    callbacks_cfg = lightning_config.callbacks
else:
    callbacks_cfg = OmegaConf.create()

if 'metrics_over_trainsteps_checkpoint' in callbacks_cfg:
    print(
        'Caution: Saving checkpoints every n train steps without deleting. This might require some free space.')
    default_metrics_over_trainsteps_ckpt_dict = {
        'metrics_over_trainsteps_checkpoint':
            {"target": 'pytorch_lightning.callbacks.ModelCheckpoint',
             'params': {
                 "dirpath": os.path.join(ckptdir, 'trainstep_checkpoints'),
                 "filename": "{epoch:06}-{step:09}",
                 "verbose": True,
                 'save_top_k': -1,
                 'every_n_train_steps': 10000,
                 'save_weights_only': True
             }
             }
    }
    default_callbacks_cfg.update(default_metrics_over_trainsteps_ckpt_dict)

callbacks_cfg = OmegaConf.merge(default_callbacks_cfg, callbacks_cfg)
if 'ignore_keys_callback' in callbacks_cfg and hasattr(trainer_opt, 'resume_from_checkpoint'):
    callbacks_cfg.ignore_keys_callback.params['ckpt_path'] = trainer_opt.resume_from_checkpoint
elif 'ignore_keys_callback' in callbacks_cfg:
    del callbacks_cfg['ignore_keys_callback']

trainer_kwargs["callbacks"] = [instantiate_from_config(callbacks_cfg[k]) for k in callbacks_cfg]


Monitoring val/loss_simple_ema as checkpoint metric.
Merged modelckpt-cfg: 
{'target': 'pytorch_lightning.callbacks.ModelCheckpoint', 'params': {'dirpath': 'logs\\2022-06-17T14-39-19_lsun_churches-ldm-kl-8\\checkpoints', 'filename': '{epoch:06}', 'verbose': True, 'save_last': True, 'monitor': 'val/loss_simple_ema', 'save_top_k': 3}}


D:\miniconda\envs\vrkit\lib\site-packages\pytorch_lightning\loggers\test_tube.py:105: LightningDeprecationWarning: The TestTubeLogger is deprecated since v1.5 and will be removed in v1.7. We recommend switching to the `pytorch_lightning.loggers.TensorBoardLogger` as an alternative.
  rank_zero_deprecation(


In [25]:
trainer_opt

Namespace(accelerator='gpu', benchmark=True, devices=1, gpus=1)

In [26]:
trainer = Trainer(accelerator="gpu") # Trainer.from_argparse_args(trainer_opt, **trainer_kwargs)
trainer.logdir = logdir  ###

D:\miniconda\envs\vrkit\lib\site-packages\pytorch_lightning\loops\utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [27]:
# data
data = instantiate_from_config(config.data)

In [28]:
config.data

{'target': 'main.DataModuleFromConfig', 'params': {'batch_size': 24, 'num_workers': 5, 'wrap': False, 'train': {'target': 'ldm.data.lsun.LSUNChurchesTrain', 'params': {'size': 256}}, 'validation': {'target': 'ldm.data.lsun.LSUNChurchesValidation', 'params': {'size': 256}}}}

In [29]:
data

In [30]:
data.prepare_data()

In [31]:
data.setup()

In [32]:
for k in data.datasets:
    print(f"{k}, {data.datasets[k].__class__.__name__}, {len(data.datasets[k])}")

train, LSUNChurchesTrain, 121227
validation, LSUNChurchesValidation, 5000


In [33]:
# configure learning rate
bs, base_lr = config.data.params.batch_size, config.model.base_learning_rate
if not cpu:
#     ngpu = len(lightning_config.trainer.gpus.strip(",").split(','))
# else:
    ngpu = 1
if 'accumulate_grad_batches' in lightning_config.trainer:
    accumulate_grad_batches = lightning_config.trainer.accumulate_grad_batches
else:
    accumulate_grad_batches = 1
print(f"accumulate_grad_batches = {accumulate_grad_batches}")
lightning_config.trainer.accumulate_grad_batches = accumulate_grad_batches
if opt.scale_lr:
    model.learning_rate = accumulate_grad_batches * ngpu * bs * base_lr
    print(
        "Setting learning rate to {:.2e} = {} (accumulate_grad_batches) * {} (num_gpus) * {} (batchsize) * {:.2e} (base_lr)".format(
            model.learning_rate, accumulate_grad_batches, ngpu, bs, base_lr))
else:
    model.learning_rate = base_lr
    print("++++ NOT USING LR SCALING ++++")
    print(f"Setting learning rate to {model.learning_rate:.2e}")

accumulate_grad_batches = 1
Setting learning rate to 1.20e-03 = 1 (accumulate_grad_batches) * 1 (num_gpus) * 24 (batchsize) * 5.00e-05 (base_lr)


In [34]:
# # allow checkpointing via USR1
# def melk(*args, **kwargs):
#     # run all checkpoint hooks
#     if trainer.global_rank == 0:
#         print("Summoning checkpoint.")
#         ckpt_path = os.path.join(ckptdir, "last.ckpt")
#         trainer.save_checkpoint(ckpt_path)


# def divein(*args, **kwargs):
#     if trainer.global_rank == 0:
#         import pudb;
#         pudb.set_trace()


# import signal

# signal.signal(signal.SIGUSR1, melk)
# signal.signal(signal.SIGUSR2, divein)

In [35]:
opt.train = True

In [36]:
trainer.logdir 

'logs\\2022-06-17T14-39-19_lsun_churches-ldm-kl-8'

# Start training

In [ ]:
trainer.fit(model, data)

D:\miniconda\envs\vrkit\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:326: LightningDeprecationWarning: Base `LightningModule.on_train_batch_start` hook signature has changed in v1.5. The `dataloader_idx` argument will be removed in v1.7.
  rank_zero_deprecation(
Missing logger folder: D:\research\latent-diffusion\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type             | Params
-------------------------------------------------------
0 | model             | DiffusionWrapper | 294 M 
1 | model_ema         | LitEma           | 0     
2 | first_stage_model | AutoencoderKL    | 83.7 M
-------------------------------------------------------
294 M     Trainable params
83.7 M    Non-trainable params
378 M     Total params
1,514.483 Total estimated model params size (MB)


Setting up LambdaLR scheduler...


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

### USING STD-RESCALING ###
setting self.scale_factor to 0.21411871910095215
### USING STD-RESCALING ###


D:\miniconda\envs\vrkit\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\result.py:229: UserWarning: You called `self.log('global_step', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(


In [ ]:
# train_dataloader = data.train_dataloader()

# batch = next(iter(train_dataloader))

# batch['image'].shape

# model.device

# model.device